# **Baseline measurements**

A baseline, non-federated model trained on the whole MNIST dataset.

Parameters are set as in the federated clients.

In [1]:
import sys

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F

sys.path.append("..")
import tools
import net as net_implementation

from tqdm.notebook import tqdm

In [2]:
DEVICE = torch.device("cuda:0" if (torch.cuda.is_available() and tools.is_mem_enough()) else "cpu")

free cuda memory:	11.300171


In [3]:
DATA_PATH = "../../data/MNIST/mnist_train.csv"
RESULT_PATH = "../../results/baseline/"

## Data loader

In [4]:
train_loader, test_loader = tools.create_data_loaders(df_path=DATA_PATH)

## Training

In [5]:
EPOCHS = 500

In [6]:
net = net_implementation.Net().to(DEVICE)
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=.9)
train_losses = []
test_correct, test_accuracy, test_loss = [], [], []
for _ in tqdm(range(EPOCHS)):
    #training:
    l_, total = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        loss = F.nll_loss(net(images), labels)
        l_ += loss.item()
        total += labels.size(0)
        loss.backward()
        optimizer.step()
    train_losses.append(l_)
    #testing:
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = net(images)
            loss += F.nll_loss(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    test_correct.append(correct)
    test_accuracy.append(accuracy)
    test_loss.append(loss)

  0%|          | 0/500 [00:00<?, ?it/s]

/mnt/data/phd/kutatas/FeLeSh/src/2_measurements/../net.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


## Saving results

In [7]:
metrics = []
values = []
rounds = []
epochs = []

for i, (train_loss, test_loss, test_accuracy) in enumerate(zip(train_losses, test_loss, test_accuracy)):
    metrics.append("test_loss")
    rounds.append(-1)
    epochs.append(i)
    values.append(test_loss)
    metrics.append("train_loss")
    rounds.append(-1)
    epochs.append(i)
    values.append(train_loss*len(test_loader)/len(train_loader))
    metrics.append("test_accuracy")
    rounds.append(-1)
    epochs.append(i)
    values.append(test_accuracy)

In [8]:
df_save = pd.DataFrame(data = np.array([metrics, values, rounds, epochs]).T,
                      columns = ["metric", "value", "round", "epoch"])
df_save.to_csv(RESULT_PATH+"performance.csv", index=False)